In [47]:
import pandas as pd
import numpy as np
from collections import Counter


In [48]:
df = pd.read_csv("wikidetox/toxicity_annotated_comments.tsv",  sep='\t')
df_labels = pd.read_csv("wikidetox/toxicity_annotations.tsv",  sep='\t')

In [49]:
df_labels.drop(['worker_id'],axis=1, inplace=True)
df.drop(['year','logged_in','ns','sample'],axis=1, inplace=True)


In [50]:
df['comment'] = df['comment'].str.replace('NEWLINE_TOKEN','\n')

In [51]:
df

,rev_id,comment,split
0,2232.0,This:\n:One can make an analogy in mathematica...,train
1,4216.0,`\n\n:Clarification for you (and Zundark's ri...,train
2,8953.0,Elected or Electoral? JHK,test
3,26547.0,`This is such a fun entry. Devotchka\n\nI on...,train
4,28959.0,Please relate the ozone hole to increases in c...,test
...,...,...,...
159681,699848324.0,`\n\n\nThese sources don't exactly exude a sen...,train
159682,699851288.0,\n\nThe Institute for Historical Review is a p...,test
159683,699857133.0,\n:The way you're trying to describe it in thi...,dev
159684,699891012.0,\n\n== Warning ==\n\nThere is clearly a protec...,train


In [52]:
dev = df[df['split']=='dev']
train = df[df['split']=='train']
test = df[df['split']=='test']

In [53]:
mydic = Counter()
counterdic = Counter()
for index, row in df_labels.iterrows():
    mydic[row['rev_id']] += row['toxicity']
    counterdic[row['rev_id']] += 1

In [54]:
true_labels = Counter()
for k in mydic.keys():
    if mydic[k]/counterdic[k] > 0.500000:
        true_labels[k] = 1
    else:
        true_labels[k] = 0

In [55]:
len([k for k in true_labels.keys() if true_labels[k] !=0])/len(mydic.keys())


0.09620129504151897

In [56]:
def add_label(df):
    mylist = []
    for index,row in df.iterrows():
        mylist.append(true_labels[row['rev_id']])
    df['labels'] = mylist
add_label(dev)
add_label(test)
add_label(train)

/home/anishthite/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [57]:
dev[dev['labels']==1].count() / dev.count()

rev_id     0.095524
comment    0.095524
split      0.095524
labels     0.095524
dtype: float64

In [58]:
train[train['labels']==1].count() / train.count()

rev_id     0.096612
comment    0.096612
split      0.096612
labels     0.096612
dtype: float64

In [59]:
test[test['labels']==1].count() / test.count()

rev_id     0.095651
comment    0.095651
split      0.095651
labels     0.095651
dtype: float64

In [60]:
dev.drop(['split','rev_id'],axis=1, inplace=True)
train.drop(['split','rev_id'],axis=1, inplace=True)
test.drop(['split','rev_id'],axis=1, inplace=True)

In [61]:
traintest = pd.concat([train,test])


In [62]:
traintest

,comment,labels
0,This:\n:One can make an analogy in mathematica...,0
1,`\n\n:Clarification for you (and Zundark's ri...,0
3,`This is such a fun entry. Devotchka\n\nI on...,0
6,`\n\n\nI fixed the link; I also removed ``home...,0
7,`If they are ``indisputable`` then why does th...,0
...,...,...
159659,\n\n== These are the files on this topic that ...,0
159661,"\n\nYeah, I realized I created a duplicate ID....",0
159677,"`\nYes, from the word ``Guci`` or ``Puci`` mea...",0
159682,\n\nThe Institute for Historical Review is a p...,0


In [63]:
traintest.to_csv('traintest.csv',index=False,header=None)
test.to_csv('test.csv',index=False,header=None)
dev.to_csv('val.csv',index=False,header=None)

In [27]:
!gzip traintest.csv
!gzip test.csv
!gzip val.csv

gzip: traintest.csv.gz already exists; do you wish to overwrite (y or n)? ^C
gzip: test.csv.gz already exists; do you wish to overwrite (y or n)? ^C
gzip: val.csv.gz already exists; do you wish to overwrite (y or n)? ^C
